In [1]:
# Instalación base de Qiskit (incluye qiskit.circuit, qiskit.quantum_info, etc.)
!pip install qiskit

# Instalación de los Estimadores y Samplers locales que has usado (qiskit.primitives)
# Esto ya viene en qiskit, pero se recomienda asegurar el paquete de simulación.
!pip install qiskit-aer

# Instalación de la librería para operar con hardware real o simuladores en la nube de IBM
# (Necesaria si usas Session, EstimatorV2, SamplerV2)
!pip install qiskit-ibm-runtime

# Instalación de las herramientas para optimización y QAOA (para usar el QAOAAnsatz)
# Esto generalmente viene con qiskit, pero es bueno asegurarse si da errores
!pip install qiskit-terra

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 368.7/368.7 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 MB 17.3 MB/s eta 0:00:00


In [3]:
# Uninstall any older/conflicting versions
!pip uninstall qiskit qiskit-terra qiskit-opflow -y

# Install Qiskit 1.0+, Qiskit-Aer (simulators), and Qiskit-IBM-Runtime
!pip install qiskit qiskit-aer qiskit-ibm-runtime numpy scipy matplotlib

Found existing installation: qiskit 2.2.2
Uninstalling qiskit-2.2.2:
  Successfully uninstalled qiskit-2.2.2
Found existing installation: qiskit-terra 0.46.3
Uninstalling qiskit-terra-0.46.3:
  Successfully uninstalled qiskit-terra-0.46.3
  Using cached qiskit-2.2.2-cp39-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (12 kB)
Using cached qiskit-2.2.2-cp39-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (8.0 MB)


In [6]:
from qiskit.quantum_info import SparsePauliOp
import numpy as np

def generate_tsp_qubo(cost_matrix: np.ndarray, A: float = 1.0, B: float = 10.0) -> SparsePauliOp:
    """
    Genera el Hamiltoniano QUBO (SparsePauliOp) para el Problema del Viajante de Comercio (TSP).

    Utiliza la formulación QUBO/Ising estándar, codificada en operadores Pauli Z.

    Args:
        cost_matrix: Matriz de costos/distancias entre ciudades (N x N).
        A: Coeficiente para el término de Costo (A).
        B: Coeficiente para el término de Restricción (B), B >> A.

    Returns:
        SparsePauliOp: Hamiltoniano QUBO/Ising total del TSP.
    """
    N = cost_matrix.shape[0]  # Número de ciudades
    total_qubits = N * N      # Qubits = Ciudades * Posiciones

    # 1. Inicializar el Hamiltoniano total a cero (I con coeficiente 0)
    H_total = SparsePauliOp("I" * total_qubits, coeffs=[0.0])

    # Función auxiliar para generar un operador Pauli Z en un índice dado
    def get_z_op(idx: int) -> SparsePauliOp:
        label = ['I'] * total_qubits
        label[total_qubits - 1 - idx] = 'Z' # Qiskit usa orden LSB primero
        return SparsePauliOp("".join(label), coeffs=[1.0])

    # Función auxiliar para generar un operador Pauli Z Z en dos índices
    def get_zz_op(idx1: int, idx2: int) -> SparsePauliOp:
        label = ['I'] * total_qubits
        label[total_qubits - 1 - idx1] = 'Z'
        label[total_qubits - 1 - idx2] = 'Z'
        return SparsePauliOp("".join(label), coeffs=[1.0])

    # -----------------------------------------------------------
    # Conversión QUBO a Ising: x_i = (1 - Z_i) / 2
    # x_i^2 = x_i
    # x_i x_k = 1/4 (I - Z_i - Z_k + Z_i Z_k)
    # -----------------------------------------------------------

    # 2. TÉRMINO DE COSTO (Minimizar Distancia: H_costo)
    H_costo = SparsePauliOp("I" * total_qubits, coeffs=[0.0])

    for j in range(N):
        j_next = (j + 1) % N
        for i in range(N):
            for k in range(N):
                if i == k: continue

                weight = cost_matrix[i, k]
                idx_ij = i * N + j
                idx_k_jnext = k * N + j_next

                # Término: x_{i, j} * x_{k, j_next} * w_{i, k}
                # Mapeo a Pauli: (1/4) * (I - Z_a - Z_b + Z_a Z_b)

                term_zz = get_zz_op(idx_ij, idx_k_jnext)
                term_z1 = get_z_op(idx_ij)
                term_z2 = get_z_op(idx_k_jnext)
                term_i = SparsePauliOp("I" * total_qubits, coeffs=[1.0])

                H_costo += (term_i - term_z1 - term_z2 + term_zz) * (weight / 4)

    # 3. TÉRMINOS DE RESTRICCIÓN (H_restricción)
    H_restriccion = SparsePauliOp("I" * total_qubits, coeffs=[0.0])

    # H_restriccion = B * (H_pos + H_ciudad)

    # Restricción: Posición Única (Cada posición j debe tener UNA ciudad)
    # H_pos = sum_j (1 - sum_i x_{i, j})^2
    for j in range(N):
        # En la forma Ising, la restricción de 'igual a 1' es:
        # Penalización: (sum_i Z_{i, j})^2
        sum_z = SparsePauliOp("I" * total_qubits, coeffs=[0.0])
        for i in range(N):
            idx_ij = i * N + j
            sum_z += get_z_op(idx_ij)

        H_restriccion += sum_z.compose(sum_z)

    # Restricción: Ciudad Única (Cada ciudad i debe estar en UNA posición)
    # H_ciudad = sum_i (1 - sum_j x_{i, j})^2
    for i in range(N):
        sum_z = SparsePauliOp("I" * total_qubits, coeffs=[0.0])
        for j in range(N):
            idx_ij = i * N + j
            sum_z += get_z_op(idx_ij)

        H_restriccion += sum_z.compose(sum_z)

    # 4. HAMILTONIANO TOTAL
    # Utilizar la sobrecarga de operadores para suma y multiplicación escalar
    H_total = (A * H_costo + B * H_restriccion).simplify()

    return H_total

# =================================================================
# EJEMPLO DE USO
# =================================================================

# 4 ciudades (N=4) con distancias arbitrarias
COSTOS_EJEMPLO = np.array([
    [ 0, 10,  8, 12],
    [10,  0, 15,  5],
    [ 8, 15,  0,  9],
    [12,  5,  9,  0]
])

A_PARAM = 1.0  # Peso del costo
B_PARAM = 50.0 # Peso de la restricción (B >> A)

# Generar el Hamiltoniano
tsp_hamiltonian = generate_tsp_qubo(COSTOS_EJEMPLO, A=A_PARAM, B=B_PARAM)

# Assuming tsp_hamiltonian has been correctly generated
# ...

print(f"Número de qubits necesarios: {tsp_hamiltonian.num_qubits} (N*N = 4*4 = 16)")
print(f"Número de términos Pauli: {len(tsp_hamiltonian)}")
print("\nPrimeros 5 términos del Hamiltoniano QUBO/Ising:")

# 🚨 CORRECTED LINE
print(tsp_hamiltonian[:5])

Número de qubits necesarios: 16 (N*N = 4*4 = 16)
Número de términos Pauli: 113

Primeros 5 términos del Hamiltoniano QUBO/Ising:
SparsePauliOp(['IIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIZ', 'IIIIIIIIIIZIIIII', 'IIIIIIIIIIZIIIIZ', 'IIIIIIZIIIIIIIII'],
              coeffs=[1718. +0.j,  -15. +0.j,  -15. +0.j,    2.5+0.j,  -16. +0.j])


Explicación de la Salida del Hamiltoniano
TSP1. Número de Qubits Necesarios: 16 (NN = 44 = 16)Significado:
Esto confirma que tu formulación QUBO utilizó el esquema de codificación estándar del TSP, donde se requiere una variable binaria $x_{i, j}$ para cada ciudad $i$ en cada posición $j$.Cálculo: Para $N=4$ ciudades, necesitas $N^2 = 4 \times 4 = 16$ variables binarias. Por lo tanto, el Hamiltoniano requiere 16 qubits para ser representado.2. Número de Términos Pauli: 113Significado: Este es el total de operadores Pauli únicos ($I, Z, ZZ$, etc.) que conforman el Hamiltoniano $H$.Origen: El Hamiltoniano $H$ está compuesto por:Términos Cuadráticos ($Z_i Z_j$): Provienen de las penalizaciones de restricción (e.g., $(x_i + x_j - 1)^2$) y del término de costo ($x_{i, j} x_{k, j+1}$).Términos Lineales ($Z_i$): Provienen de la linealización de los términos cuadráticos y de los términos de costo originales (al sustituir $x_i = (1-Z_i)/2$).Término Constante ($I$): Resulta de la expansión de todos los términos cuadráticos.Un número grande de términos (113 en este caso) es típico para problemas QUBO/Ising, incluso para un grafo pequeño como 4 ciudades.3. Primeros 5 Términos del Hamiltoniano QUBO/IsingEsta es una muestra de los primeros cinco operadores Pauli (etiquetas) y sus coeficientes complejos.Etiqueta PauliCoeficiente (Coef)Tipo de TérminoSignificadoIIIIIIIIIIIIIIII1718. + 0.jConstante ($I^{\otimes 16}$)Este es el término constante (energía de offset) del Hamiltoniano. Resulta de la expansión de todas las penalizaciones y el costo, y se suma a la energía total.IIIIIIIIIIIIIIIZ-15. + 0.jLineal ($Z_0$)Término que afecta solo al último qubit (índice 0). Este qubit representa $x_{3, 3}$ (ciudad 3, posición 3) o $x_{0, 0}$ (ciudad 0, posición 0), dependiendo de tu convención de indexación.IIIIIIIIIIZIIIII-15. + 0.jLineal ($Z_5$)Término que afecta al qubit en la quinta posición (índice 5, contando desde 0).IIIIIIIIIIZIIIIZ2.5 + 0.jCuadrático ($Z_0 Z_5$)Término de acoplamiento entre el qubit 0 y el qubit 5. Este es el corazón de la formulación QUBO/Ising y codifica una penalización/costo entre dos variables $x_0$ y $x_5$.IIIIIIZIIIIIIIII-16. + 0.jLineal ($Z_9$)Término que afecta al qubit en la novena posición (índice 9).Nota sobre la Lectura de Etiquetas:Qiskit ordena las etiquetas Pauli de izquierda a derecha desde el qubit de mayor índice ($q_{15}$) hasta el qubit de menor índice ($q_0$).II...IZ significa que solo $q_0$ (el último qubit) es $Z$.El coeficiente (1718.) es muy grande y positivo, lo cual es normal. La energía mínima del Hamiltoniano (el resultado del TSP) será la energía de ground state y se encontrará cerca de ese valor base, pero no será $1718$.

1. Estructura y TamañoElementoValorSignificadoQubits Necesarios16 ($N^2$)
Confirma el esquema de codificación estándar del TSP: $N=4$ ciudades $\times$ $N=4$ posiciones = 16 variables binarias, requiriendo 16 qubits.Términos Pauli113El número total de operadores únicos (términos) que componen la función de costo ($H$). Este alto número es típico debido a la expansión de todas las restricciones cuadráticas.

2. Análisis de los Primeros 5 TérminosEl Hamiltoniano es una suma de operadores Pauli, donde cada término codifica una interacción (costo o penalización) entre las variables binarias del problema.
Etiqueta PauliCoeficiente (Coef)Tipo de InteracciónSignificado en el ProblemaIIIIIIIIIIIIIIII1718. + 0.jConstante ($\mathbf{I^{\otimes 16}}$)Energía de Offset. Es el valor base o punto de partida de la función de costo. La energía mínima real del TSP (el ground state del Hamiltoniano) se encontrará cerca de este valor.IIIIIIIIIIIIIIIZ-15. + 0.jLineal ($\mathbf{Z_0}$)Sesgo local (bias) que afecta al primer qubit ($q_0$). Este término lineal se origina principalmente de la expansión de las restricciones y del término de costo linealizado, impulsando la variable $x_0$ hacia 1 o 0.IIIIIIIIIIZIIIII-15. + 0.jLineal ($\mathbf{Z_5}$)Similar al anterior, es un sesgo que afecta al qubit en el índice 5 ($q_5$).IIIIIIIIIIZIIIIZ2.5 + 0.jCuadrático ($\mathbf{Z_0 Z_5}$)Acoplamiento (Penalización/Costo) entre el qubit 0 y el qubit 5. Este es el corazón de la formulación QUBO: establece una relación de costo o penalización si ambas variables binarias $x_0$ y $x_5$ son 1 simultáneamente.IIIIIIZIIIIIIIII-16. + 0.jLineal ($\mathbf{Z_9}$)Sesgo que afecta al qubit en el índice 9 ($q_9$).

Nota sobre la Lectura de EtiquetasEn Qiskit, las etiquetas Pauli se leen de izquierda a derecha del qubit de mayor índice ($\mathbf{q_{15}}$) al qubit de menor índice ($\mathbf{q_0}$).La etiqueta II...IZ significa que todos los qubits desde $q_{15}$ hasta $q_1$ tienen el operador Identidad ($I$), y solo el qubit $q_0$ tiene el operador $\mathbf{Z}$.El objetivo de este Hamiltoniano es que, al encontrar su estado fundamental (la energía mínima), el estado cuántico resultante revele la secuencia de bits (el camino TSP) que hace que las penalizaciones se anulen y el costo de la distancia se minimice.